# [문제]
- ChatPromptTemplate() : 역할 부여
- InMemoryChatMessageHistory()
- RunnableWithMessageHistory()
- chain 구성

## [문제] PromptTemplate 내 변수 설정: Chain

template = '''
당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.

이름: {name}
경력: {experience}
기술: {skills}
'''


In [1]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv


load_dotenv()

llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_completion_tokens=150,
)


In [14]:
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	# ('user', '이름: {name}\n경력: {experience}\n기술: {skills}')
	('user', '{input}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)


from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

def get_history():
    return history

from langchain_core.runnables.history import RunnableWithMessageHistory

chain = prompt | llm

with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history'
)

with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_history at 0x00000191E903A200>, history_messages_key='chat_history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [16]:
'''
'name': '박보검',
'experience': '웹 개발 신입',
'skills': 'Python, HTML, CSS, LLM'
'''

ai_message = with_message_history.invoke({
    'input': '''name: 박보검\nexperience: 웹 개발 신입\nskills: Python, HTML, CSS, LLM''' 
})

박보검 이력서

개인 정보
이름: 박보검

경력 요약
신입 웹 개발자로서 다양한 프로젝트와 학습 경험을 통해 웹 개발의 기본기를 갖추었으며, Python, HTML, CSS 등 핵심 기술에 대한 이해도를 바탕으로 실무에 빠르게 적응할 수 있는 역량을 보유하고 있습니다. 최신 AI 기술인 LLM(대형 언어 모델)에 대한 관심과 학습을 지속하며, 혁신적인 서비스 개발에 기여하고자 합니다.

전문 기술
- 프로그래밍 언어: Python
- 웹 개발: HTML, CSS
- 인공지능/머신러닝: 대형 언어

In [17]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='name: 박보검\nexperience: 웹 개발 신입\nskills: Python, HTML, CSS, LLM', additional_kwargs={}, response_metadata={}), AIMessage(content='박보검 이력서\n\n개인 정보\n이름: 박보검\n\n경력 요약\n신입 웹 개발자로서 다양한 프로젝트와 학습 경험을 통해 웹 개발의 기본기를 갖추었으며, Python, HTML, CSS 등 핵심 기술에 대한 이해도를 바탕으로 실무에 빠르게 적응할 수 있는 역량을 보유하고 있습니다. 최신 AI 기술인 LLM(대형 언어 모델)에 대한 관심과 학습을 지속하며, 혁신적인 서비스 개발에 기여하고자 합니다.\n\n전문 기술\n- 프로그래밍 언어: Python\n- 웹 개발: HTML, CSS\n- 인공지능/머신러닝: 대형 언어', additional_kwargs={}, response_metadata={'finish_reason': 'length', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-6228b613-73bd-40af-9868-7b818b59d01b-0')])